## Working with Transformers in the HuggingFace Ecosystem

In this laboratory exercise we will learn how to work with the HuggingFace ecosystem to adapt models to new tasks. As you will see, much of what is required is *investigation* into the inner-workings of the HuggingFace abstractions. With a little work, a little trial-and-error, it is fairly easy to get a working adaptation pipeline up and running.

### Exercise 1: Sentiment Analysis (warm up)

In this first exercise we will start from a pre-trained BERT transformer and build up a model able to perform text sentiment analysis. Transformers are complex beasts, so we will build up our pipeline in several explorative and incremental steps.

#### Exercise 1.1: Dataset Splits and Pre-trained model
There are a many sentiment analysis datasets, but we will use one of the smallest ones available: the [Cornell Rotten Tomatoes movie review dataset](cornell-movie-review-data/rotten_tomatoes), which consists of 5,331 positive and 5,331 negative processed sentences from the Rotten Tomatoes movie reviews.

**Your first task**: Load the dataset and figure out what splits are available and how to get them. Spend some time exploring the dataset to see how it is organized. Note that we will be using the [HuggingFace Datasets](https://huggingface.co/docs/datasets/en/index) library for downloading, accessing, splitting, and batching data for training and evaluation.

In [ ]:
import torch
print(torch.cuda.is_available())  # Deve restituire: True


True


In [ ]:
!pip install -U datasets huggingface_hub fsspec

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [ ]:
from datasets import load_dataset, get_dataset_split_names
from transformers import DataCollatorWithPadding, AutoTokenizer

# Load the dataset
dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes")

# Figure out what splits are available and how to get them
print("Splits available:", list(dataset.keys()))

# Explore the dataset
print("\nExample from the training split:")
print(dataset["train"][0])

print("\nDataset structure:")
print(dataset)


Splits available: ['train', 'validation', 'test']

Example from the training split:
{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1}

Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})


#### Exercise 1.2: A Pre-trained BERT and Tokenizer

The model we will use is a *very* small BERT transformer called [Distilbert](https://huggingface.co/distilbert/distilbert-base-uncased) this model was trained (using self-supervised learning) on the same corpus as BERT but using the full BERT base model as a *teacher*.

**Your next task**: Load the Distilbert model and corresponding tokenizer. Use the tokenizer on a few samples from the dataset and pass the tokens through the model to see what outputs are provided. I suggest you use the [`AutoModel`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html) class (and the `from_pretrained()` method) to load the model and `AutoTokenizer` to load the tokenizer).

In [ ]:
from transformers import AutoTokenizer, AutoModel

# Carica il tokenizer e il modello Distilbert pre-allenato
#il tokenizer aggiunge anche un token cls=classe prima della sequenza. riassume la frase.
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

print("Tokenizer e modello Distilbert caricati con successo!")

# Esempi dal dataset (prendiamone un paio dallo split di training)
example_texts = [dataset["train"][0]["text"], dataset["train"][1]["text"]]
print("\nEsempi di testo dal dataset:")
print(example_texts)

# Tokenizza gli esempi di testo
# Usiamo return_tensors="pt" per ottenere output come tensori PyTorch, utili per il modello
tokenized_inputs = tokenizer(example_texts, padding=True, truncation=True, return_tensors="pt")

print("\nInput tokenizzati (con padding e troncamento):")
print(tokenized_inputs)

# Passa i token attraverso il modello
# Disattiviamo i gradienti perché non stiamo addestrando qui, vogliamo solo l'output del modello
import torch
with torch.no_grad():
    model_outputs = model(**tokenized_inputs)

print("\nOutput del modello:")
# Gli output di AutoModel tipicamente contengono attributi come 'last_hidden_state'
# che sono i vettori di rappresentazione dell'ultimo layer del modello.
print("Tipo di output:", type(model_outputs))
print("Chiavi nell'output:", model_outputs.keys())
print("Forma dell'ultimo stato nascosto:", model_outputs.last_hidden_state.shape)#2 frasi nel batch, 52 token per frase (lunghezza seq), 768 dim uscita modello per ogni token. ha info su contesto.

Tokenizer e modello Distilbert caricati con successo!

Esempi di testo dal dataset:
['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .']

Input tokenizzati (con padding e troncamento):
{'input_ids': tensor([[  101,  1996,  2600,  2003, 16036,  2000,  2022,  1996,  7398,  2301,
          1005,  1055,  2047,  1000, 16608,  1000,  1998,  2008,  2002,  1005,
          1055,  2183,  2000,  2191,  1037, 17624,  2130,  3618,  2084,  7779,
         29058,  8625, 13327,  1010,  3744,  1011, 18856, 19513,  3158,  5477,
          4168,  2030,  7112, 16562,  2140,  1012,   102,     0,     0,     0,
             0,     0],
        [  101,  19

#### Exercise 1.3: A Stable Baseline

In this exercise I want you to:
1. Use Distilbert as a *feature extractor* to extract representations of the text strings from the dataset splits;
2. Train a classifier (your choice, by an SVM from Scikit-learn is an easy choice).
3. Evaluate performance on the validation and test splits.

These results are our *stable baseline* -- the **starting** point on which we will (hopefully) improve in the next exercise.

**Hint**: There are a number of ways to implement the feature extractor, but probably the best is to use a [feature extraction `pipeline`](https://huggingface.co/tasks/feature-extraction). You will need to interpret the output of the pipeline and extract only the `[CLS]` token from the *last* transformer layer. *How can you figure out which output that is?*

In [ ]:
#praticamente dobbiamo prendere i valore dell'ultimo hidden state del modello hilbert.
#ciò produce un una rappresentazione raffinata di ogni token iniziale.
#oltre ai suoi token ogni frase ha un token speciale che alla fine riassume il significato della frase.
#farò knoledge distillation lavorando proprio su questi token.

In [ ]:
# Assumiamo di avere l'output del modello da prima: model_outputs
# model_outputs.last_hidden_state ha forma [batch_size, sequence_length, hidden_size]
# Esempio: torch.Size([2, 52, 768])

# Estraiamo la rappresentazione del token [CLS]
# Usiamo lo slicing dei tensori:
# [ : ,   0,   : ]
#   ^     ^    ^
#   |     |    |
#   |     |    Seleziona tutte le dimensioni del vettore di rappresentazione (768)
#   |     |
#   |     Seleziona il token all'indice 0 nella dimensione della sequenza (che è il [CLS] token)
#   |
#   Seleziona tutti gli elementi nel batch (quindi otterremo un risultato per ogni frase nel batch)

cls_representations = model_outputs.last_hidden_state[:, 0, :]

print("\nRappresentazioni [CLS] estratte:")
# La forma sarà ora [batch_size, hidden_size]
# Esempio: torch.Size([2, 768])
print("Forma delle rappresentazioni [CLS]:", cls_representations.shape)
print("Esempio di rappresentazione [CLS] per la prima frase (primi 10 valori):", cls_representations[0][:10].tolist()) # Convertiamo in lista per una stampa più leggibile


Rappresentazioni [CLS] estratte:
Forma delle rappresentazioni [CLS]: torch.Size([2, 768])
Esempio di rappresentazione [CLS] per la prima frase (primi 10 valori): [-0.03317328914999962, -0.016808928921818733, 0.019411858171224594, -0.025717630982398987, -0.13796712458133698, -0.39617061614990234, 0.38299697637557983, 0.5117570757865906, 0.023082124069333076, -0.0555349737405777]


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

# Carica il tokenizer e il modello Distilbert pre-allenato (se non già caricati)
# Assicurati che il modello sia sul dispositivo corretto (CPU o GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if 'tokenizer' not in locals() or 'model' not in locals() or model.device != device:
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    model = AutoModel.from_pretrained("distilbert-base-uncased").to(device) # Sposta il modello sulla GPU
    print(f"Tokenizer e modello Distilbert caricati e spostati su: {device}.")


def extract_cls_features_batched(examples):
    """
    Tokenizza un batch di testi, passa i token attraverso il modello Distilbert sulla GPU,
    estrae le rappresentazioni del token [CLS] per il batch e restituisce un dizionario
    contenente le rappresentazioni e le etichette.

    Args:
        examples (dict): Un dizionario con liste per ogni campo (es. {'text': [...], 'label': [...]}).

    Returns:
        dict: Un dizionario con liste o array NumPy per 'cls_features' e 'label' per il batch.
    """
    # Tokenizza il batch di testi
    # Usiamo padding=True e truncation=True e return_tensors="pt"
    inputs = tokenizer(examples['text'], padding=True, truncation=True, return_tensors="pt", max_length=512)

    # Sposta i tensori di input sulla GPU
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Passa i token attraverso il modello per ottenere gli output
    # Usiamo torch.no_grad() perché non stiamo addestrando
    with torch.no_grad():
        outputs = model(**inputs)

    # Estrai le rappresentazioni del token [CLS] (indice 0) per l'intero batch
    # Sposta il risultato sulla CPU e converti in array NumPy
    cls_representations = outputs.last_hidden_state[:, 0, :].cpu().numpy()

    # Restituisci un dizionario con le rappresentazioni e le etichette (come liste o array)
    return {'cls_features': cls_representations, 'label': examples['label']}

print("Funzione 'extract_cls_features_batched' definita/modificata con successo.")

Tokenizer e modello Distilbert caricati e spostati su: cuda.
Funzione 'extract_cls_features_batched' definita/modificata con successo.


In [ ]:
# Applica la funzione a tutti gli split del dataset in batch
# Modificato batched=True e aggiunto batch_size
# La funzione extract_cls_features_batched accetta ora batch di esempi.
dataset_with_features = dataset.map(extract_cls_features_batched, batched=True, batch_size=32, remove_columns=['text']) # Puoi sperimentare con la dimensione del batch

print("\nDataset con feature estratte (in batch, sulla GPU):")
print(dataset_with_features)

# Ispeziona un esempio dal nuovo dataset per verificare le colonne e la forma delle feature
print("\nEsempio dal dataset con feature estratte (split 'train'):")
print(dataset_with_features["train"][0])

# Verifica la forma delle feature estratte per assicurarti che siano come previsto (hidden_size)
# Corretto: Rimosso .shape perché l'output per un singolo esempio è un array NumPy 1D
print("\nForma delle feature estratte per il primo esempio:", len(dataset_with_features["train"][0]['cls_features']))

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]


Dataset con feature estratte (in batch, sulla GPU):
DatasetDict({
    train: Dataset({
        features: ['label', 'cls_features'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['label', 'cls_features'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['label', 'cls_features'],
        num_rows: 1066
    })
})

Esempio dal dataset con feature estratte (split 'train'):
{'label': 1, 'cls_features': [-0.03317366540431976, -0.016809647902846336, 0.01941177062690258, -0.025717660784721375, -0.1379670351743698, -0.39617007970809937, 0.38299715518951416, 0.5117574334144592, 0.023082125931978226, -0.05553486943244934, -0.0631648376584053, -0.13681618869304657, -0.0517975278198719, 0.49828919768333435, 0.23182538151741028, 0.23795634508132935, -0.3116884231567383, 0.2472493201494217, 0.227940171957016, 0.04644673690199852, -0.15396276116371155, -0.1512853056192398, 0.1738920509815216, -0.07235698401927948, 0.0587148442864418, -0.1852137297391891

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Prepara i dati per l'addestramento, la validazione e il test
# Estrai le features e le etichette come array NumPy dai dataset splits
X_train = np.array(dataset_with_features["train"]["cls_features"])
y_train = np.array(dataset_with_features["train"]["label"])

X_val = np.array(dataset_with_features["validation"]["cls_features"])
y_val = np.array(dataset_with_features["validation"]["label"])

X_test = np.array(dataset_with_features["test"]["cls_features"])
y_test = np.array(dataset_with_features["test"]["label"])

print("Dati preparati per l'addestramento e la valutazione.")
print(f"Forma di X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Forma di X_val: {X_val.shape}, y_val: {y_val.shape}")
print(f"Forma di X_test: {X_test.shape}, y_test: {y_test.shape}")

Dati preparati per l'addestramento e la valutazione.
Forma di X_train: (8530, 768), y_train: (8530,)
Forma di X_val: (1066, 768), y_val: (1066,)
Forma di X_test: (1066, 768), y_test: (1066,)


In [ ]:
# Inizializza e addestra un classificatore SVM
# Usiamo un kernel lineare per semplicità e velocità
svm_classifier = SVC(kernel='linear')

print("\nAddestramento del classificatore SVM...")
svm_classifier.fit(X_train, y_train)
print("Addestramento completato.")


Addestramento del classificatore SVM...
Addestramento completato.


In [ ]:
# Valuta il classificatore sui set di validation e test
print("\nValutazione sul set di validation...")
y_pred_val = svm_classifier.predict(X_val)
accuracy_val = accuracy_score(y_val, y_pred_val)
print(f"Accuratezza sul set di validation: {accuracy_val:.4f}")

print("\nValutazione sul set di test...")
y_pred_test = svm_classifier.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
print(f"Accuratezza sul set di test: {accuracy_test:.4f}")

print("\nBaseline stabilita.")


Valutazione sul set di validation...
Accuratezza sul set di validation: 0.8189

Valutazione sul set di test...
Accuratezza sul set di test: 0.8068

Baseline stabilita.


In [ ]:
#nel dataset sono salvati: split, etichette, featres (CLS)
from google.colab import drive
drive.mount('/content/drive')
save_path = "/content/drive/MyDrive/dataset_cls_features"
dataset_with_features.save_to_disk(save_path)
print(f"Dataset salvato in: {save_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving the dataset (0/1 shards):   0%|          | 0/8530 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1066 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset salvato in: /content/drive/MyDrive/dataset_cls_features


-----
### Exercise 2: Fine-tuning Distilbert

In this exercise we will fine-tune the Distilbert model to (hopefully) improve sentiment analysis performance.

#### Exercise 2.1: Token Preprocessing

The first thing we need to do is *tokenize* our dataset splits. Our current datasets return a dictionary with *strings*, but we want *input token ids* (i.e. the output of the tokenizer). This is easy enough to do my hand, but the HugginFace `Dataset` class provides convenient, efficient, and *lazy* methods. See the documentation for [`Dataset.map`](https://huggingface.co/docs/datasets/v3.5.0/en/package_reference/main_classes#datasets.Dataset.map).

**Tip**: Verify that your new datasets are returning for every element: `text`, `label`, `intput_ids`, and `attention_mask`.

In [ ]:
# Funzione di tokenizzazione
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

# Applica la tokenizzazione ai vari split del dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Verifica che i campi richiesti siano presenti
example = tokenized_datasets["train"][0]
print("Chiavi presenti nell'esempio:", example.keys())
assert all(key in example for key in ["text", "label", "input_ids", "attention_mask"]), "Alcune chiavi mancano!"
print("Verifica completata: il dataset ha tutti i campi richiesti.")


Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Chiavi presenti nell'esempio: dict_keys(['text', 'label', 'input_ids', 'attention_mask'])
Verifica completata: il dataset ha tutti i campi richiesti.


#### Exercise 2.2: Setting up the Model to be Fine-tuned

In this exercise we need to prepare the base Distilbert model for fine-tuning for a *sequence classification task*. This means, at the very least, appending a new, randomly-initialized classification head connected to the `[CLS]` token of the last transformer layer. Luckily, HuggingFace already provides an `AutoModel` for just this type of instantiation: [`AutoModelForSequenceClassification`](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelforsequenceclassification). You will want you instantiate one of these for fine-tuning.

In [ ]:
from transformers import AutoModelForSequenceClassification

# Definisci il numero di classi (2 per sentiment: positivo / negativo)
num_labels = 2

# Carica DistilBERT con una testa di classificazione (pronta per fine-tuning)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels
)

print("Modello pronto per il fine-tuning:")
print(model)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modello pronto per il fine-tuning:
DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropo

#### Exercise 2.3: Fine-tuning Distilbert

Finally. In this exercise you should use a HuggingFace [`Trainer`](https://huggingface.co/docs/transformers/main/en/trainer) to fine-tune your model on the Rotten Tomatoes training split. Setting up the trainer will involve (at least):


1. Instantiating a [`DataCollatorWithPadding`](https://huggingface.co/docs/transformers/en/main_classes/data_collator) object which is what *actually* does your batch construction (by padding all sequences to the same length).
2. Writing an *evaluation function* that will measure the classification accuracy. This function takes a single argument which is a tuple containing `(logits, labels)` which you should use to compute classification accuracy (and maybe other metrics like F1 score, precision, recall) and return a `dict` with these metrics.  
3. Instantiating a [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.51.1/en/main_classes/trainer#transformers.TrainingArguments) object using some reasonable defaults.
4. Instantiating a `Trainer` object using your train and validation splits, you data collator, and function to compute performance metrics.
5. Calling `trainer.train()`, waiting, waiting some more, and then calling `trainer.evaluate()` to see how it did.

**Tip**: When prototyping this laboratory I discovered the HuggingFace [Evaluate library](https://huggingface.co/docs/evaluate/en/index) which provides evaluation metrics. However I found it to have insufferable layers of abstraction and getting actual metrics computed. I suggest just using the Scikit-learn metrics...

In [ ]:
# ⬇️ 1. Librerie e setup
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# 🔗 Collega Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 📥 2. Carica il dataset e il tokenizer
dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# ✏️ 3. Tokenizza il dataset
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize, batched=True)

# 🧱 4. Data collator per padding dinamico
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 🧠 5. Modello con testa di classificazione
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
)

# 📊 6. Funzione per il calcolo delle metriche
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=1
)


# 🚀 8. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# 🏋️‍♂️ 9. Addestramento del modello
trainer.train()

# 🧪 10. Valutazione finale sul test set
results = trainer.evaluate(tokenized_datasets["test"])
print("\n📊 Risultati sul test set:")
for k, v in results.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")

# 💾 11. Salvataggio del modello e tokenizer su Google Drive
save_path = "/content/drive/MyDrive/distilbert-finetuned-rotten"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)
print(f"\n✅ Modello e tokenizer salvati in: {save_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-20-1583363447.py:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: john17 (john17-universit-di-firenze) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,0.566200
200,0.404200
300,0.399800
400,0.379100
500,0.357600
600,0.305400
700,0.230600
800,0.228200
900,0.256000
1000,0.239800



📊 Risultati sul test set:
eval_loss: 0.5524
eval_accuracy: 0.8377
eval_precision: 0.8249
eval_recall: 0.8574
eval_f1: 0.8408
eval_runtime: 4.0880
eval_samples_per_second: 260.7650
eval_steps_per_second: 16.3900
epoch: 3.0000

✅ Modello e tokenizer salvati in: /content/drive/MyDrive/distilbert-finetuned-rotten


#confronto tra SVM e Fine-tuning DistilBERT

In [ ]:
import pandas as pd
from IPython.display import display

# 🔹 Risultati del modello SVM (già calcolati)
svm_val_acc = accuracy_val
svm_test_acc = accuracy_test

# 🔹 Risultati del modello DistilBERT fine-tunato
bert_val_results = trainer.evaluate(tokenized_datasets["validation"])
bert_test_results = trainer.evaluate(tokenized_datasets["test"])

bert_val_acc = bert_val_results["eval_accuracy"]
bert_test_acc = bert_test_results["eval_accuracy"]

# 📊 Crea la tabella di confronto
df_comparison = pd.DataFrame({
    "Modello": ["SVM (CLS features)", "DistilBERT fine-tuned"],
    "Accuracy validation": [svm_val_acc, bert_val_acc],
    "Accuracy test": [svm_test_acc, bert_test_acc]
})

# ✅ Visualizza la tabella in Colab
print("📊 Confronto modelli:")
display(df_comparison)

# 💾 Salva la tabella su Google Drive come CSV
save_path = "/content/drive/MyDrive/confronto_modelli.csv"
df_comparison.to_csv(save_path, index=False)
print(f"\n✅ Tabella salvata in: {save_path}")


📊 Confronto modelli:


,Modello,Accuracy validation,Accuracy test
0,SVM (CLS features),0.818949,0.806754
1,DistilBERT fine-tuned,0.850844,0.837711



✅ Tabella salvata in: /content/drive/MyDrive/confronto_modelli.csv


-----
### Exercise 3: Choose at Least One


#### Exercise 3.1: Efficient Fine-tuning for Sentiment Analysis (easy)

In Exercise 2 we fine-tuned the *entire* Distilbert model on Rotten Tomatoes. This is expensive, even for a small model. Find an *efficient* way to fine-tune Distilbert on the Rotten Tomatoes dataset (or some other dataset).

**Hint**: You could check out the [HuggingFace PEFT library](https://huggingface.co/docs/peft/en/index) for some state-of-the-art approaches that should "just work". How else might you go about making fine-tuning more efficient without having to change your training pipeline from above?

In [ ]:
!pip install -U peft accelerate bitsandbytes


#carica moedllo più configura lora

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType
import torch

# 1. Carica il modello base
base_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

# 2. Configurazione LoRA
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_lin", "v_lin"]
)


# 3. Applica LoRA al modello
peft_model = get_peft_model(base_model, lora_config)

# 4. Stampa i parametri addestrabili (verifica)
peft_model.print_trainable_parameters()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925


#Inserisci il modello peft_model nel Trainer

In [ ]:
trainer = Trainer(
    model=peft_model,  # Usa il modello con LoRA
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()


/tmp/ipython-input-23-2343072534.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
100,0.685300
200,0.652100
300,0.578400
400,0.483700
500,0.446600
600,0.457500
700,0.407700
800,0.435200
900,0.451100
1000,0.431400


{'eval_loss': 0.4073343873023987,
 'eval_accuracy': 0.8151969981238274,
 'eval_precision': 0.8333333333333334,
 'eval_recall': 0.7879924953095685,
 'eval_f1': 0.8100289296046287,
 'eval_runtime': 4.0637,
 'eval_samples_per_second': 262.322,
 'eval_steps_per_second': 16.487,
 'epoch': 3.0}

#salviamo tutto

In [ ]:
import pandas as pd

# 📁 1. Definisci percorso di salvataggio su Google Drive
save_path = "/content/drive/MyDrive/distilbert-lora-rotten"

# 💾 2. Salva il modello fine-tunato (solo i pesi LoRA)
# Questo include solo i parametri modificati durante il fine-tuning (≈1% del modello)
peft_model.save_pretrained(save_path)

# 💾 3. Salva anche il tokenizer per garantire compatibilità futura
# Serve per tokenizzare input esattamente come durante il training
tokenizer.save_pretrained(save_path)

print(f"✅ Modello LoRA e tokenizer salvati in: {save_path}")

# 📊 4. Salva le metriche finali in un CSV
# Queste metriche ti permettono di documentare le prestazioni del modello
metrics = {
    "model": ["distilbert-lora"],
    "trainable_params": [739586],
    "all_params": [67694596],
    "trainable_percent": [round(739586 / 67694596 * 100, 4)],
    "eval_accuracy": [0.8152],
    "eval_f1": [0.8100],
    "eval_precision": [0.8333],
    "eval_recall": [0.7880]
}

df_metrics = pd.DataFrame(metrics)
csv_path = f"{save_path}/lora_metrics.csv"
df_metrics.to_csv(csv_path, index=False)

print(f"✅ Metriche salvate in: {csv_path}")


✅ Modello LoRA e tokenizer salvati in: /content/drive/MyDrive/distilbert-lora-rotten
✅ Metriche salvate in: /content/drive/MyDrive/distilbert-lora-rotten/lora_metrics.csv


#### Exercise 3.2: Fine-tuning a CLIP Model (harder)

Use a (small) CLIP model like [`openai/clip-vit-base-patch16`](https://huggingface.co/openai/clip-vit-base-patch16) and evaluate its zero-shot performance on a small image classification dataset like ImageNette or TinyImageNet. Fine-tune (using a parameter-efficient method!) the CLIP model to see how much improvement you can squeeze out of it.

**Note**: There are several ways to adapt the CLIP model; you could fine-tune the image encoder, the text encoder, or both. Or, you could experiment with prompt learning.

**Tip**: CLIP probably already works very well on ImageNet and ImageNet-like images. For extra fun, look for an image classification dataset with different image types (e.g. *sketches*).

In [ ]:
# Your code here.

#### Exercise 3.3: Choose your Own Adventure

There are a *ton* of interesting and fun models on the HuggingFace hub. Pick one that does something interesting and adapt it in some way to a new task. Or, combine two or more models into something more interesting or fun. The sky's the limit.

**Note**: Reach out to me by email or on the Discord if you are unsure about anything.

# Task
Utilizza Distilbert come feature extractor per estrarre le rappresentazioni testuali dal dataset, addestra un classificatore (es. SVM) su queste feature e valuta le sue prestazioni sugli split di validation e test per stabilire una baseline.

## Preparare i dati per l'estrazione delle feature

### Subtask:
Definire una funzione per tokenizzare e preparare gli esempi del dataset, includendo l'estrazione della rappresentazione [CLS].


**Reasoning**:
Definire la funzione `extract_cls_features` per tokenizzare il testo, passare i token attraverso il modello Distilbert ed estrarre la rappresentazione del token [CLS].



**Reasoning**:
Applicare la funzione `extract_cls_features` a tutti gli split del dataset per ottenere un nuovo dataset contenente le feature estratte e le etichette.



**Reasoning**:
La funzione `extract_cls_features` deve restituire un dizionario per poter essere utilizzata con `dataset.map`. Modificare la funzione per restituire un dizionario con le feature e l'etichetta.



## Addestramento e Valutazione del Classificatore Baseline

### Subtask:
Preparare i dati per l'addestramento e la valutazione.

### Subtask:
Addestrare un classificatore SVM.

### Subtask:
Valutare le prestazioni del classificatore sui set di validation e test.

## Riepilogo del Progetto: Sentiment Analysis con Distilbert

Questo progetto esplora l'adattamento di modelli Transformer per la sentiment analysis, utilizzando il dataset Rotten Tomatoes.

### Obiettivo:
Valutare due approcci per la sentiment analysis:
1.  Utilizzare Distilbert come feature extractor seguito da un classificatore tradizionale (baseline).
2.  Fine-tunare il modello Distilbert per il task di classificazione del sentiment.

### Dataset:
[Cornell Rotten Tomatoes movie review dataset](https://huggingface.co/datasets/rotten_tomatoes)
*   Split disponibili: `train`, `validation`, `test`
*   Struttura dati: `{ 'text': '...', 'label': ... }`
*   Dimensione split: Train (8530), Validation (1066), Test (1066)

### Modello Base:
[Distilbert (distilbert-base-uncased)](https://huggingface.co/distilbert/distilbert-base-uncased)

### Baseline (SVM su Feature CLS):
*   Approccio: Utilizzo di Distilbert per estrarre la rappresentazione del token `[CLS]` come feature, seguita da un classificatore SVM lineare.
*   Risultati:
    *   Accuratezza sul set di validation: [Incollare risultato da cella MVnuFZ0JKzBz - Accuracy sul set di validation]
    *   Accuratezza sul set di test: [Incollare risultato da cella MVnuFZ0JKzBz - Accuracy sul set di test]

### Fine-tuning (Distilbert con Trainer):
*   Approccio: Fine-tuning end-to-end di Distilbert con una testa di classificazione utilizzando il `Trainer` di Hugging Face.
*   Iperparametri principali:
    *   Epoche: 3
    *   Dimensione Batch (Train/Eval): 16
    *   Learning Rate: 2e-5
*   Risultati sul set di test:
    *   eval_loss: [Incollare risultato da cella ab17178d-5028-47e3-af97-953e8de5aae4 - eval_loss]
    *   eval_accuracy: [Incollare risultato da cella ab17178d-5028-47e3-af97-953e8de5aae4 - eval_accuracy]
    *   eval_precision: [Incollare risultato da cella ab17178d-5028-47e3-af97-953e8fde5aae4 - eval_precision]
    *   eval_recall: [Incollare risultato da cella ab17178d-5028-47e3-af97-953e8de5aae4 - eval_recall]
    *   eval_f1: [Incollare risultato da cella ab17178d-5028-47e3-af97-953e8de5aae4 - eval_f1]

### Osservazioni:
*   [Aggiungere qui le tue osservazioni sui risultati, sul confronto tra baseline e fine-tuning, sull'uso della GPU, ecc.]
*   Il fine-tuning con il `Trainer` ha mostrato un miglioramento significativo rispetto alla baseline (come previsto).
*   L'estrazione delle feature in batch sulla GPU è più efficiente rispetto all'estrazione singola.

### File Salvati (su Google Drive):
*   Dataset con feature CLS (per baseline): `/content/drive/MyDrive/dataset_cls_features`
*   Modello fine-tunato e Tokenizer: `/content/drive/MyDrive/distilbert-finetuned-rotten`

In [ ]:
import json
import os
from google.colab import drive

# Assicurati che Drive sia montato
drive.mount('/content/drive', force_remount=True)

# Definire il percorso di salvataggio su Google Drive
save_dir = "/content/drive/MyDrive/project_results"
os.makedirs(save_dir, exist_ok=True) # Crea la cartella se non esiste

# Nome del file per i risultati
results_filename = "fine_tuning_test_results.json"
results_path = os.path.join(save_dir, results_filename)

# Assicurati che la variabile 'results' esista (viene creata dalla cella di fine-tuning)
# Puoi aggiungere un controllo per sicurezza, ma per ora assumiamo che la cella di fine-tuning sia stata eseguita
# if 'results' in locals():

# Salva il dizionario dei risultati in un file JSON
with open(results_path, 'w') as f:
    json.dump(results, f, indent=4)

print(f"Risultati del fine-tuning sul set di test salvati in: {results_path}")

# else:
#     print("Variabile 'results' non trovata. Eseguire prima la cella di fine-tuning.")